In [30]:
import findspark
findspark.init('/home/kant/spark-2.4.4-bin-hadoop2.7')

In [31]:
from pyspark.sql import SparkSession

In [32]:
spark = SparkSession.builder.appName('NLP').getOrCreate()

In [33]:
from pyspark.ml.feature import Tokenizer, RegexTokenizer

In [34]:
from pyspark.sql.functions import col, udf

In [35]:
from pyspark.sql.types import IntegerType

In [36]:
sen_df = spark.createDataFrame([
    (0,"I love my Data"),
    (1, "I know about the Spark"),
    (2, "I wish java could use case classes"),
    (3, "Logistis,regression, models are neat")
],['id','sentences'])

In [37]:
sen_df.show()

+---+--------------------+
| id|           sentences|
+---+--------------------+
|  0|      I love my Data|
|  1|I know about the ...|
|  2|I wish java could...|
|  3|Logistis,regressi...|
+---+--------------------+



In [38]:
tokenizer = Tokenizer(inputCol='sentences',outputCol='words')

In [39]:
regex_tokenizer = RegexTokenizer(inputCol='sentences', outputCol='words', pattern='\\W')

In [40]:
count_tokens = udf(lambda words: len(words), IntegerType())

In [41]:
tokenized = tokenizer.transform(sen_df)

In [42]:
tokenized.show()

+---+--------------------+--------------------+
| id|           sentences|               words|
+---+--------------------+--------------------+
|  0|      I love my Data| [i, love, my, data]|
|  1|I know about the ...|[i, know, about, ...|
|  2|I wish java could...|[i, wish, java, c...|
|  3|Logistis,regressi...|[logistis,regress...|
+---+--------------------+--------------------+



In [43]:
tokenized.withColumn('tokens',count_tokens(col('words'))).show()

+---+--------------------+--------------------+------+
| id|           sentences|               words|tokens|
+---+--------------------+--------------------+------+
|  0|      I love my Data| [i, love, my, data]|     4|
|  1|I know about the ...|[i, know, about, ...|     5|
|  2|I wish java could...|[i, wish, java, c...|     7|
|  3|Logistis,regressi...|[logistis,regress...|     4|
+---+--------------------+--------------------+------+



In [44]:
rg_tokenized = regex_tokenizer.transform(sen_df)

In [45]:
rg_tokenized.withColumn('tokens',count_tokens(col('words'))).show()

+---+--------------------+--------------------+------+
| id|           sentences|               words|tokens|
+---+--------------------+--------------------+------+
|  0|      I love my Data| [i, love, my, data]|     4|
|  1|I know about the ...|[i, know, about, ...|     5|
|  2|I wish java could...|[i, wish, java, c...|     7|
|  3|Logistis,regressi...|[logistis, regres...|     5|
+---+--------------------+--------------------+------+



In [46]:
from pyspark.ml.feature import StopWordsRemover

In [47]:
sentence_dataframe = spark.createDataFrame([
    (0,['I','saw','the','green','horse']),
    (1,['I','Love','my','job'])
],['id','tokens'])

In [48]:
sentence_dataframe.show()

+---+--------------------+
| id|              tokens|
+---+--------------------+
|  0|[I, saw, the, gre...|
|  1|  [I, Love, my, job]|
+---+--------------------+



In [49]:
remover = StopWordsRemover(inputCol='tokens', outputCol='filtered')

In [51]:
remover.transform(sentence_dataframe).show()

+---+--------------------+-------------------+
| id|              tokens|           filtered|
+---+--------------------+-------------------+
|  0|[I, saw, the, gre...|[saw, green, horse]|
|  1|  [I, Love, my, job]|        [Love, job]|
+---+--------------------+-------------------+



# n-gram

In [53]:
from pyspark.ml.feature import NGram

In [54]:
wordDataFrame = spark.createDataFrame([
    (0, ["Hi", "I", "heard", "about", "Spark"]),
    (1, ["I", "wish", "Java", "could", "use", "case", "classes"]),
    (2, ["Logistic", "regression", "models", "are", "neat"])
], ["id", "words"])


In [55]:
ngram = NGram(n=2, inputCol='words',outputCol='grams')

In [57]:
ngram.transform(wordDataFrame).show(truncate =False)

+---+------------------------------------------+------------------------------------------------------------------+
|id |words                                     |grams                                                             |
+---+------------------------------------------+------------------------------------------------------------------+
|0  |[Hi, I, heard, about, Spark]              |[Hi I, I heard, heard about, about Spark]                         |
|1  |[I, wish, Java, could, use, case, classes]|[I wish, wish Java, Java could, could use, use case, case classes]|
|2  |[Logistic, regression, models, are, neat] |[Logistic regression, regression models, models are, are neat]    |
+---+------------------------------------------+------------------------------------------------------------------+

